# Find bragg stack parameters from normal reflection spectrum
Using TMM to generate reflectance with varying the parameters (thickness, RI) in a loop and calculate the least standard deviation with experimental data to get the best fit.

In [ ]:
import matplotlib.pyplot as plt
from phc_codes.phc_utils import refractive_index
import phc_codes.spectra_utils as su
from phc_codes.TMM import PhCTransferMatrix
import numpy as np
%matplotlib widget

## Refractive index data

In [ ]:
n_scotch_tape = 1.5
n_quartz = (
    lambda x: (
        1
        + 0.6961663 / (1 - (0.0684043 / x) ** 2)
        + 0.4079426 / (1 - (0.1162414 / x) ** 2)
        + 0.8974794 / (1 - (9.896161 / x) ** 2)
    )
    ** 0.5
)  # Quartz #
npva = 1.457 + 0.0002j
npdms = lambda x : (1+1.0093/(1-13185.0/x**2))**.5 #1.427 + 0.0005j
npmma = lambda x : (1+0.99654/(1-0.00787/(x*1e3)**2)+0.18964/(1-0.02191/(x*1e3)**2)+0.00411/(1-3.85727/(x*1e3)**2))**.5
print(npdms(700))

## input wavelength range and data file

In [ ]:
wl1 = 380
wl2 = 990

refl_data = su.load_spectra(
        r"path_to_spectrum\cavity.TXT",          # load spectrum file
    )
theta_i = 0  # incident angle (deg)
theta_i *= np.pi / 180 # rad

smooth_data =    su.clip_wl(su.smooth_spectrum(refl_data, cut_freq=20), # 
    wl1,
    wl2
)

smooth_data[:, 1] -= min(smooth_data[:, 1])     # offset zero level
smooth_data[:, 1] /= np.max(smooth_data[:, 1])  # nortmalize reflectance
wls = smooth_data[:, 0]                         # wavelengths    


## Define the structure

In [ ]:
ni = 1.0
nsub = refractive_index("Quartz_expt")
nh = refractive_index("TiO2_expt")  # 2.41 + 0.0024j   
nl = refractive_index("SiO2_expt")  # 1.46 + 0.0010j 
n_cavity = nl

tmetal = 0

n_bilayer = 6           # number of bilayers
nn = np.array([nl, nh] * n_bilayer + [n_cavity] + [nh, nl] * n_bilayer, dtype=object)  # R.I. of layers
#tt = np.array([tmetal, tpdms] + [th, tl] * n_bilayer)

## Transfer matrix for adjusing the paramerteres
### Find best fit for the thickness ranges

In [ ]:
tls = range(94, 101)      # range of nl matrial thickness for searching
ths = range(52, 59)       # .....  nh  ......
diffs = []
diff = 100
for ttl in tls:
  for tth in ths:
    t_cavity = ttl * 4                    # cavity thickness parameter
    tt = np.array([ttl, tth] * n_bilayer + [t_cavity*0.97] + [tth, ttl] * n_bilayer) # thickness of layers
    #tt[2] = tl  / 2
    # tt[-1] = 1000  # buffer layer
    tr_matrix = PhCTransferMatrix(wls, ni, nn, nsub, tt, theta_i)
    # ax.plot(wls, tr_matrix.R)  # / np.max(tr_matrix.R))
    refl = tr_matrix.R
    refl /= np.max(refl)
    # find best fit by simply taking the summed absolute difference
    diff1 = np.std(refl - smooth_data[:, 1])
    if diff1 < diff:
      tl = ttl
      th = tth                               #<<<<<<< I/P
      diff = diff1
    #diffs += [np.std(refl - smooth_data[:, 1])]


## Redefine the thickness with the new values and calculate transfer matrix

In [ ]:
tt = np.array([tl, th] * n_bilayer + [t_cavity*0.97] + [th, tl] * n_bilayer) # thickness of layers       #<<<<<< I/P
#tt[2] = tl  / 2
# tt[6] = tl * 3 / 2
# tt[-1] = 1000  # buffer layer

print(tt)

# final calc with best fit parameters
tr_matrix = PhCTransferMatrix(wls, ni, nn, nsub, tt, theta_i)
refl = tr_matrix.R
refl /= np.max(refl)

## Plot data as needed

In [ ]:
plt.clf()
fig, ax = plt.subplots()
plt.style.use("../plt_templates/originlike_lineplot.mplstyle")

# su.plot_spectrum(ax, refl_data)
ax = su.plot_spectrum(ax, smooth_data, lw=0.5)
ax.plot(wls, refl, lw=0.5)  # / np.max(tr_matrix.R))
plt.text(200, 1.1, f"tt = {tt}")
plt.legend(['expriment', 'computation'])
plt.xlabel("wavelength (nm)")
plt.ylabel("reflectance")
#plt.text(750, 0.9, f"th={th}, tl={tl}, tpva={tpva}")
fig.savefig('./data/dip_coat_thickness.svg')
plt.show()